In [ ]:
# ┌───────────────────────────────────────────────┐
# │                    IMPORTS                    │
# └───────────────────────────────────────────────┘
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import zipfile
from PIL import Image, ImageOps
import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import v2 as transforms
from PIL import Image
# Determine Device for the whole session
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
# Uncomment if not using Personal Google Drive for Hosting Training Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip dataset
file_path="/content/drive/MyDrive/RBE595/p3/dataset/window_dataset_12000.zip"
directory_to_extract_to = "/content/dataset" # Define the extraction directory
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
image_path = "/content/dataset/window_dataset/images/"
mask_path = "/content/dataset/window_dataset/masks/"

# List files in the directories to pick one example
import os
image_files = sorted(os.listdir(image_path))
print(len(image_files))
mask_files = sorted(os.listdir(mask_path))


12000


In [ ]:
# # Check image and mask

# image_path = "/content/dataset/window_dataset/images/"
# mask_path = "/content/dataset/window_dataset/masks/"

# # List files in the directories to pick one example
# import os
# image_files = sorted(os.listdir(image_path))
# mask_files = sorted(os.listdir(mask_path))

# if image_files and mask_files:
#     first_image_file = image_files[2]
#     first_mask_file = mask_files[2]
# Aerial_P3_Unet_2.ipynb_
# [1]
# 11s

# # ┌───────────────────────────────────────────────┐
# # │                    IMPORTS                    │
# # └───────────────────────────────────────────────┘
# import matplotlib.pyplot as plt
# import torch.nn as nn
# import torch.nn.functional as F
# import torch
# import numpy as np
# from torchvision import datasets, transforms
# from torchvision.transforms import v2
# from torch.utils.data import Dataset, DataLoader
# from torch.optim import AdamW
# import zipfile
# from PIL import Image, ImageOps
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# import os
# import torch
# from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision.transforms import v2 as transforms
# from PIL import Image
# # Determine Device for the whole session
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {DEVICE}")

# Using device: cuda

# [2]
# 26s

# Mounted at /content/drive

# [ ]

# # Unzip dataset
# file_path="/content/drive/MyDrive/RBE595/p3/dataset/window_dataset_12000.zip"
# directory_to_extract_to = "/content/dataset" # Define the extraction directory
# with zipfile.ZipFile(file_path, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

# [3]
# 0s

# image_path = "/content/dataset/window_dataset/images/"
# mask_path = "/content/dataset/window_dataset/masks/"

# # List files in the directories to pick one example
# import os
# image_files = sorted(os.listdir(image_path))
# print(len(image_files))
# mask_files = sorted(os.listdir(mask_path))

# Next steps:
# [ ]
# 0s
# [ ]

# # ┌───────────────────────────────────────────────┐
# # │                 HYPERPARAMETERS               │
# # └───────────────────────────────────────────────┘
# NUM_EPOCHS = 10
# BATCH_SIZE = 8 # Reduced batch size to potentially resolve OOM error

# [ ]

# Total dataset size: 7000
# Train size: 5600
# Validation size: 700
# Test size: 700

# ==================================================
# DATALOADER INFORMATION
# ==================================================

# Train loader: 700 batches
# Validation loader: 88 batches
# Test loader: 88 batches

# /tmp/ipython-input-2431407332.py:35: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
#   A.CoarseDropout(


# Train batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])
# Val batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])
# Test batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])

# ✓ All DataLoaders created successfully!
# ==================================================

# [ ]

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # --- Attention Block (as you provided) ---
# class AttentionBlock(nn.Module):
#     def __init__(self, F_g, F_l, F_int):
#         super(AttentionBlock, self).__init__()
#         self.W_g = nn.Sequential(
#             nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(F_int)
#         )
#         self.W_x = nn.Sequential(
#             nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(F_int)
#         )
#         self.psi = nn.Sequential(
#             nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(1),
#             nn.Sigmoid()
#         )
#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, g, x):
#         g1 = self.W_g(g)
#         x1 = self.W_x(x)
#         psi = self.relu(g1 + x1)
#         psi = self.psi(psi)
#         return x * psi

# # --- Double Conv Block (as you provided) ---
# def double_conv(in_channels, out_channels):
#     '''
#     Return a Convolutional Block with Two Convolutional Layers with ReLU Activation based on in_channels and
#     out_channels, with a stride of 3 and padding of 1. If input has H and W, then output will have same H and W.
#     '''
#     return nn.Sequential(
#         nn.Conv2d(in_channels, out_channels, 3, padding=1),
#         nn.BatchNorm2d(out_channels),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(out_channels, out_channels, 3, padding=1),
#         nn.BatchNorm2d(out_channels),
#         nn.ReLU(inplace=True)
#     )

# # --- NEW: Attention U-Net Class (Modified) ---
# class UNet(nn.Module):
#     '''
#     Implementation of an Attention U-Net (3-level).
#     '''
#     def __init__(self, in_channels, out_channels):
#         super().__init__()

#         # --- Encoder (3 levels) ---
#         self.dconv_down1 = double_conv(in_channels, 64)
#         self.dconv_down2 = double_conv(64, 128)
#         self.dconv_down3 = double_conv(128, 256)
#         # Removed dconv_down4
#         self.maxpool = nn.MaxPool2d(2)

#         # --- Bottleneck ---
#         # Input from 3rd level (256 channels), output 512
#         self.bottleneck = double_conv(256, 512)

#         # --- Decoder ---
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

#         # --- Attention Blocks ---
#         # Removed att4
#         # F_g=512 (from upsample(bottleneck)), F_l=256 (from conv3)
#         self.att3 = AttentionBlock(F_g=512, F_l=256, F_int=128)
#         # F_g=256 (from dconv_up3), F_l=128 (from conv2)
#         self.att2 = AttentionBlock(F_g=256, F_l=128, F_int=64)
#         # F_g=128 (from dconv_up2), F_l=64 (from conv1)
#         self.att1 = AttentionBlock(F_g=128, F_l=64, F_int=32)

#         # --- Decoder Convolutions ---
#         # Removed dconv_up4
#         # Input: (g=512) + (att3=256) = 768
#         self.dconv_up3 = double_conv(512 + 256, 256)  # 768  -> 256
#         # Input: (g=256) + (att2=128) = 384
#         self.dconv_up2 = double_conv(256 + 128, 128)  # 384  -> 128
#         # Input: (g=128) + (att1=64) = 192
#         self.dconv_up1 = double_conv(128 + 64, 64)   # 192  -> 64

#         # --- Output Layer ---
#         self.conv_last = nn.Conv2d(64, out_channels, 1)

#     def forward(self, x):
#         '''
#         Forward Pass Method for the Attention UNet
#         '''

#         # ┌───────────────────────────────────────────────┐
#         # │         ENCODER BLOCKS (DOWNSAMPLING)         │
#         # └───────────────────────────────────────────────┘
#         conv1 = self.dconv_down1(x)                                 # (N, 64, H, W)
#         x = self.maxpool(conv1)

#         conv2 = self.dconv_down2(x)                                 # (N, 128, H/2, W/2)
#         x = self.maxpool(conv2)

#         conv3 = self.dconv_down3(x)                                 # (N, 256, H/4, W/4)
#         x = self.maxpool(conv3)                                     # (N, 256, H/8, W/8)

#         # Removed 4th Encoder Block

#         # ┌───────────────────────────────────────────────┐
#         # │         BOTTLENECK                            │
#         # └───────────────────────────────────────────────┘
#         x = self.bottleneck(x)                                      # (N, 512, H/8, W/8)

#         # ┌───────────────────────────────────────────────┐
#         # │           DECODER BLOCKS (UPSAMPLING)         │
#         # └───────────────────────────────────────────────┘

#         # --- First Decoder Block (was Second) ---
#         g = self.upsample(x)                                        # (N, 512, H/4, W/4)
#         x_skip3 = conv3                                             # (N, 256, H/4, W/4)
#         att3 = self.att3(g=g, x=x_skip3)                            # (N, 256, H/4, W/4)
#         x = torch.cat([g, att3], dim=1)                             # (N, 512 + 256, H/4, W/4)
#         x = self.dconv_up3(x)                                       # (N, 256, H/4, W/4)

#         # --- Second Decoder Block (was Third) ---
#         g = self.upsample(x)                                        # (N, 256, H/2, W/2)
#         x_skip2 = conv2                                             # (N, 128, H/2, W/2)
#         att2 = self.att2(g=g, x=x_skip2)                            # (N, 128, H/2, W/2)
#         x = torch.cat([g, att2], dim=1)                             # (N, 256 + 128, H/2, W/2)
#         x = self.dconv_up2(x)                                       # (N, 128, H/2, W/2)

#         # --- Third Decoder Block (was Fourth) ---
#         g = self.upsample(x)                                        # (N, 128, H, W)
#         x_skip1 = conv1                                             # (N, 64, H, W)
#         att1 = self.att1(g=g, x=x_skip1)                            # (N, 64, H, W)
#         x = torch.cat([g, att1], dim=1)                             # (N, 128 + 64, H, W)
#         x = self.dconv_up1(x)                                       # (N, 64, H, W)

#         # --- Final Output Layer ---
#         out = self.conv_last(x)                                     # (N, out_channels, H, W)

#         return out

# [ ]

# class CombinedLoss(nn.Module):
#     """
#     Combination of BCE, Dice, and boundary loss for clean edges
#     """
#     def __init__(self, bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2):
#         super().__init__()
#         self.bce_weight = bce_weight
#         self.dice_weight = dice_weight
#         self.boundary_weight = boundary_weight
#         self.bce = nn.BCEWithLogitsLoss()

#     def dice_loss(self, pred, target, smooth=1e-6):
#         pred = torch.sigmoid(pred)
#         intersection = (pred * target).sum(dim=(2, 3))
#         union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
#         dice = (2. * intersection + smooth) / (union + smooth)
#         return 1 - dice.mean()

#     def boundary_loss(self, pred, target):
#         """
#         Penalize predictions at boundaries more heavily
#         """
#         pred = torch.sigmoid(pred)

#         # Compute gradient of target (boundaries)
#         kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
#                                 dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
#         kernel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
#                                 dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)

#         # Detect edges in target
#         edge_x = F.conv2d(target, kernel_x, padding=1)
#         edge_y = F.conv2d(target, kernel_y, padding=1)
#         edges = torch.sqrt(edge_x**2 + edge_y**2)
#         edges = (edges > 0.5).float()

#         # Weight loss by edges (penalize errors at boundaries more)
#         boundary_error = F.binary_cross_entropy(pred, target, reduction='none')
#         boundary_error = boundary_error * (1 + 5 * edges)  # 5x penalty at edges

#         return boundary_error.mean()

#     def forward(self, pred, target):
#         bce = self.bce(pred, target)
#         dice = self.dice_loss(pred, target)
#         boundary = self.boundary_loss(pred, target)

#         total = (self.bce_weight * bce +
#                 self.dice_weight * dice +
#                 self.boundary_weight * boundary)
#         return total

# [ ]

# # Model initialisation

# model= UNet(in_channels=3, out_channels=1).to(DEVICE)
# BCE_loss = nn.BCEWithLogitsLoss()
# criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# [ ]

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# def calculate_dice_iou(inputs, targets, smooth=1e-7):
#     """
#     Calculates Dice and IoU scores for a batch.
#     Assumes inputs are binary predictions (0.0 or 1.0) and targets are binary masks.
#     """
#     # Flatten tensors
#     inputs = inputs.view(-1)
#     targets = targets.view(-1)

#     # Calculate intersection
#     intersection = (inputs * targets).sum()

#     # Calculate Dice Score
#     total_sum = inputs.sum() + targets.sum()
#     dice = (2. * intersection + smooth) / (total_sum + smooth)

#     # Calculate IoU (Jaccard Index)
#     union = total_sum - intersection
#     iou = (intersection + smooth) / (union + smooth)

#     return dice.item(), iou.item()

# [ ]


# # Training
# best_val_dice = -1  # Initialize best_val_dice to a low value

# for epoch in range(NUM_EPOCHS):
#   train_loss=0

#   model.train()
#   for images, mask in train_loader:
#     images=images.to(DEVICE)
#     mask=mask.to(DEVICE)
#     optimizer.zero_grad()
#     output=model(images)
#     # bce_loss=BCE_loss(output, mask)
#     # dice_loss_value=dice_loss(output, mask)
#     # loss=0.5*bce_loss+0.5*dice_loss_value
#     loss=criterion(output, mask)

#     loss.backward()
#     optimizer.step()
#     train_loss+=loss.item()

#   avg_train_loss = train_loss / len(train_loader)
# # validation
#   model.eval()
#   tatal_loss=0
#   total_dice_loss=0
#   total_bce_loss=0
#   total_dice_score=0
#   total_iou_score=0
#   total_val_loss=0
#   with torch.no_grad():
#     for images, mask in val_loader:

#       images=images.to(DEVICE)
#       mask=mask.to(DEVICE)
#       output=model(images)
#       # bce = BCE_loss(output, mask)
#       # dice_val_loss = dice_loss(output, mask)
#       # loss = 0.5 * bce + 0.5 * dice_val_loss
#       loss=criterion(output, mask)
#       total_val_loss += loss.item()
#       probs=torch.sigmoid(output)
#       binary_preds = (probs > 0.5).float()
#       dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#       total_dice_score += dice_score
#       total_iou_score += iou_score

#     # Calculate average metrics for the epoch
#     avg_val_loss = total_val_loss / len(val_loader)
#     avg_val_dice = total_dice_score / len(val_loader)
#     avg_val_iou = total_iou_score / len(val_loader)

#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
#           f"Train Loss: {avg_train_loss:.4f}.. "
#           f"Val Loss: {avg_val_loss:.4f}.. "
#           f"Val Dice: {avg_val_dice:.4f}.. "
#           f"Val IoU: {avg_val_iou:.4f}")

#     # --- Save the best model ---
#     if 'best_val_dice' in locals() and avg_val_dice > best_val_dice:
#         best_val_dice = avg_val_dice
#         torch.save(model.state_dict(), 'UNet_background_attention_1.pth')
#         print(f"New best model saved with Dice: {avg_val_dice:.4f}")

# Epoch 1/10.. Train Loss: 0.1810.. Val Loss: 0.0342.. Val Dice: 0.9704.. Val IoU: 0.9427
# New best model saved with Dice: 0.9704
# Epoch 2/10.. Train Loss: 0.1020.. Val Loss: 0.0277.. Val Dice: 0.9736.. Val IoU: 0.9487
# New best model saved with Dice: 0.9736
# Epoch 3/10.. Train Loss: 0.0860.. Val Loss: 0.0306.. Val Dice: 0.9674.. Val IoU: 0.9372
# Epoch 4/10.. Train Loss: 0.0833.. Val Loss: 0.0214.. Val Dice: 0.9782.. Val IoU: 0.9574
# New best model saved with Dice: 0.9782
# Epoch 5/10.. Train Loss: 0.0747.. Val Loss: 0.0210.. Val Dice: 0.9785.. Val IoU: 0.9580
# New best model saved with Dice: 0.9785
# Epoch 6/10.. Train Loss: 0.0692.. Val Loss: 0.0162.. Val Dice: 0.9831.. Val IoU: 0.9668
# New best model saved with Dice: 0.9831
# Epoch 7/10.. Train Loss: 0.0637.. Val Loss: 0.0181.. Val Dice: 0.9805.. Val IoU: 0.9618
# Epoch 8/10.. Train Loss: 0.0586.. Val Loss: 0.0169.. Val Dice: 0.9818.. Val IoU: 0.9643
# Epoch 9/10.. Train Loss: 0.0553.. Val Loss: 0.0233.. Val Dice: 0.9718.. Val IoU: 0.9454
# Epoch 10/10.. Train Loss: 0.0552.. Val Loss: 0.0166.. Val Dice: 0.9829.. Val IoU: 0.9664

# [ ]

# # --- Testing ---
# print("\nTraining Finished. Loading best model for testing.")
# model.load_state_dict(torch.load('UNet_background_attention_1.pth')) # Load best weights
# model.eval()

# total_test_dice = 0
# total_test_iou = 0

# with torch.no_grad():
#     for images, mask in test_loader:
#         images = images.to(DEVICE)
#         mask = mask.to(DEVICE)
#         output = model(images)

#         probs = torch.sigmoid(output)
#         binary_preds = (probs > 0.5).float()

#         dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#         total_test_dice += dice_score
#         total_test_iou += iou_score

# avg_test_dice = total_test_dice / len(test_loader)
# avg_test_iou = total_test_iou / len(test_loader)

# print(f"--- Final Test Metrics ---")
# print(f"Test Dice Score: {avg_test_dice:.4f}")
# print(f"Test IoU Score:  {avg_test_iou:.4f}")


# Training Finished. Loading best model for testing.
# --- Final Test Metrics ---
# Test Dice Score: 0.9829
# Test IoU Score:  0.9665

# [ ]

# import torch
# import os
# import torchvision.utils as vutils # Import for saving image grids

# # --- Testing ---
# print("\nTraining Finished. Loading best model for testing.")
# model.load_state_dict(torch.load('UNet_background_attention_1.pth')) # Load best weights
# model.eval()

# total_test_dice = 0
# total_test_iou = 0

# # --- Additions for saving output ---
# output_dir = "test_outputs"
# os.makedirs(output_dir, exist_ok=True)
# print(f"Saving output masks to {output_dir}/")
# # -------------------------------------


# with torch.no_grad():
#     # Add 'i' for enumeration to get unique filenames
#     for i, (images, mask) in enumerate(test_loader):
#         images = images.to(DEVICE)
#         mask = mask.to(DEVICE)
#         output = model(images)

#         probs = torch.sigmoid(output)
#         binary_preds = (probs > 0.5).float()

#         dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#         total_test_dice += dice_score
#         total_test_iou += iou_score

#         # --- Save the output masks ---
#         # This saves the input, ground truth, and prediction for this batch
#         # 'vutils.save_image' automatically creates a grid if your batch size > 1

#         # Save the original input images
#         vutils.save_image(images,
#                           os.path.join(output_dir, f"batch_{i:03d}_input.png"),
#                           normalize=True) # Normalize for proper viewing

#         # Save the ground truth masks
#         vutils.save_image(mask,
#                           os.path.join(output_dir, f"batch_{i:03d}_ground_truth.png"))

#         # Save your model's predictions
#         vutils.save_image(binary_preds,
#                           os.path.join(output_dir, f"batch_{i:03d}_prediction.png"))
#         # -----------------------------

# avg_test_dice = total_test_dice / len(test_loader)
# avg_test_iou = total_test_iou / len(test_loader)

# print(f"--- Final Test Metrics ---")
# print(f"Test Dice Score: {avg_test_dice:.4f}")
# print(f"Test IoU Score:  {avg_test_iou:.4f}")

# print(f"\nFinished testing. All output images saved to '{output_dir}'.")


# Training Finished. Loading best model for testing.
# Saving output masks to test_outputs/
# --- Final Test Metrics ---
# Test Dice Score: 0.9829
# Test IoU Score:  0.9665

# Finished testing. All output images saved to 'test_outputs'.

# [ ]


# Start coding or generate with AI.

# Colab paid products - Cancel contracts here


#     image = plt.imread(os.path.join(image_path, first_image_file))
#     mask = plt.imread(os.path.join(mask_path, first_mask_file))

#     plt.imshow(image)
#     plt.title("Example Image")
#     plt.show()

#     plt.imshow(mask, cmap='gray') # Use gray colormap for masks
#     plt.title("Example Mask")
#     plt.show()
# else:
#     print("No image or mask files found in the specified directories.")

In [ ]:
# ┌───────────────────────────────────────────────┐
# │                 HYPERPARAMETERS               │
# └───────────────────────────────────────────────┘
NUM_EPOCHS = 10
BATCH_SIZE = 4 # Reduced batch size to potentially resolve OOM error

In [ ]:
class CustomDataset(Dataset):
    '''
    This dataset loads an image and its corresponding segmentation mask
    using Albumentations for robust augmentation.
    '''
    def __init__(self, image_dir, mask_dir, is_train=True):
        self.image_dir = image_dir
        self.mask_dir = mask_dir

        self.image_files = sorted(os.listdir(self.image_dir))
        self.mask_files = sorted(os.listdir(self.mask_dir))

        if is_train:
            # 1. Training transforms (Merged List)
            self.transform = A.Compose([
                # --- Essential (Keep at top) ---
                A.Resize(height=640, width=640, p=1.0),

                # --- Spatial Augmentations ---
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),      # Updated p=0.5
                A.Rotate(limit=45, p=0.5),    # ADDED

                # --- Color Augmentations ---
                A.ToGray(p=0.1), # Kept from previous version
                A.RandomBrightnessContrast(p=0.3), # Updated

                # --- Noise, Blur, and Edge Augmentations ---
                A.GaussNoise(p=0.4),               # Updated
                A.GaussianBlur(blur_limit=(3, 7), p=0.4), # ADDED
                A.MotionBlur(blur_limit=7, p=0.3),      # ADDED
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.4), # ADDED

                # --- Dropout Augmentation ---
                A.CoarseDropout(
                    max_holes=8, max_height=20, max_width=20, # Updated
                    min_holes=3, min_height=5, min_width=5,
                    fill_value=0, p=0.4
                ),

                # --- Final conversion (Keep at bottom) ---
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                ),
                ToTensorV2() # Converts image and mask to PyTorch tensors
            ])
        else:
            # 2. Validation/Test transforms (Unchanged)
            self.transform = A.Compose([
                A.Resize(height=640, width=640, p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                ),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        mask_name = self.mask_files[idx]

        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = Image.open(img_path).convert("RGB")
        image = ImageOps.exif_transpose(image) # Auto-orientation

        mask = Image.open(mask_path).convert("L")

        image_np = np.array(image)
        mask_np = np.array(mask)

        # Ensure mask is binary (0 or 1)
        mask_np = (mask_np > 128).astype(np.uint8) * 255 # Assuming mask is grayscale with values > 128 as foreground

        transformed = self.transform(image=image_np, mask=mask_np)

        image_tensor = transformed['image']
        mask_tensor = transformed['mask'].unsqueeze(0).float() / 255.0 # Normalize mask to 0-1

        return image_tensor, mask_tensor

# --- Create Train, Validation, and Test DataLoaders ---

# Define your paths and parameters
image_path = "/content/dataset/window_dataset/images/"
mask_path = "/content/dataset/window_dataset/masks/"


# Define split ratios
TRAIN_RATIO = 0.8  # 80% for training
VAL_RATIO = 0.1   # 10% for validation
TEST_RATIO = 0.1  # 10% for testing

# Create the full dataset (with training augmentation for now)
full_dataset = CustomDataset(image_path, mask_path, is_train=True)

# Calculate split sizes
dataset_size = len(full_dataset)
train_size = int(TRAIN_RATIO * dataset_size)
val_size = int(VAL_RATIO * dataset_size)
test_size = dataset_size - train_size - val_size  # Remaining goes to test

print(f"Total dataset size: {dataset_size}")
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")
print(f"Test size: {test_size}")

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

# Update the is_train flag for validation and test datasets
# Create new dataset instances for val and test without augmentation
val_dataset_no_aug = CustomDataset(image_path, mask_path, is_train=False)
test_dataset_no_aug = CustomDataset(image_path, mask_path, is_train=False)

# Get the same indices from the split
val_indices = val_dataset.indices
test_indices = test_dataset.indices

# Create subsets with no augmentation
from torch.utils.data import Subset
val_dataset = Subset(val_dataset_no_aug, val_indices)
test_dataset = Subset(test_dataset_no_aug, test_indices)

# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # Shuffle training data
    num_workers=2,  # Parallel data loading
    pin_memory=True  # Faster data transfer to GPU
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Don't shuffle validation data
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Don't shuffle test data
    num_workers=2,
    pin_memory=True
)

# Print DataLoader information
print("\n" + "="*50)
print("DATALOADER INFORMATION")
print("="*50)
print(f"\nTrain loader: {len(train_loader)} batches")
print(f"Validation loader: {len(val_loader)} batches")
print(f"Test loader: {len(test_loader)} batches")

# Verify shapes
try:
    train_images, train_masks = next(iter(train_loader))
    print(f"\nTrain batch - Images shape: {train_images.shape}, Masks shape: {train_masks.shape}")

    val_images, val_masks = next(iter(val_loader))
    print(f"Val batch - Images shape: {val_images.shape}, Masks shape: {val_masks.shape}")

    test_images, test_masks = next(iter(test_loader))
    print(f"Test batch - Images shape: {test_images.shape}, Masks shape: {test_masks.shape}")

    print("\n✓ All DataLoaders created successfully!")
    print("="*50)
except Exception as e:
    print(f"\nError loading batches: {e}")

Total dataset size: 12000
Train size: 9600
Validation size: 1200
Test size: 1200

DATALOADER INFORMATION

Train loader: 2400 batches
Validation loader: 300 batches
Test loader: 300 batches


/tmp/ipython-input-2475501229.py:35: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(



Train batch - Images shape: torch.Size([4, 3, 640, 640]), Masks shape: torch.Size([4, 1, 640, 640])
Val batch - Images shape: torch.Size([4, 3, 640, 640]), Masks shape: torch.Size([4, 1, 640, 640])
Test batch - Images shape: torch.Size([4, 3, 640, 640]), Masks shape: torch.Size([4, 1, 640, 640])

✓ All DataLoaders created successfully!


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Attention Block (as you provided) ---
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

# --- Double Conv Block (as you provided) ---
def double_conv(in_channels, out_channels):
    '''
    Return a Convolutional Block with Two Convolutional Layers with ReLU Activation based on in_channels and
    out_channels, with a stride of 3 and padding of 1. If input has H and W, then output will have same H and W.
    '''
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

# --- NEW: Attention U-Net Class (Modified) ---
class UNet(nn.Module):
    '''
    Implementation of an Attention U-Net (3-level).
    '''
    def __init__(self, in_channels, out_channels):
        super().__init__()

        # --- Encoder (3 levels) ---
        self.dconv_down1 = double_conv(in_channels, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        # Removed dconv_down4
        self.maxpool = nn.MaxPool2d(2)

        # --- Bottleneck ---
        # Input from 3rd level (256 channels), output 512
        self.bottleneck = double_conv(256, 512)

        # --- Decoder ---
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # --- Attention Blocks ---
        # Removed att4
        # F_g=512 (from upsample(bottleneck)), F_l=256 (from conv3)
        self.att3 = AttentionBlock(F_g=512, F_l=256, F_int=128)
        # F_g=256 (from dconv_up3), F_l=128 (from conv2)
        self.att2 = AttentionBlock(F_g=256, F_l=128, F_int=64)
        # F_g=128 (from dconv_up2), F_l=64 (from conv1)
        self.att1 = AttentionBlock(F_g=128, F_l=64, F_int=32)

        # --- Decoder Convolutions ---
        # Removed dconv_up4
        # Input: (g=512) + (att3=256) = 768
        self.dconv_up3 = double_conv(512 + 256, 256)  # 768  -> 256
        # Input: (g=256) + (att2=128) = 384
        self.dconv_up2 = double_conv(256 + 128, 128)  # 384  -> 128
        # Input: (g=128) + (att1=64) = 192
        self.dconv_up1 = double_conv(128 + 64, 64)   # 192  -> 64

        # --- Output Layer ---
        self.conv_last = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        '''
        Forward Pass Method for the Attention UNet
        '''

        # ┌───────────────────────────────────────────────┐
        # │         ENCODER BLOCKS (DOWNSAMPLING)         │
        # └───────────────────────────────────────────────┘
        conv1 = self.dconv_down1(x)                                 # (N, 64, H, W)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)                                 # (N, 128, H/2, W/2)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)                                 # (N, 256, H/4, W/4)
        x = self.maxpool(conv3)                                     # (N, 256, H/8, W/8)

        # Removed 4th Encoder Block

        # ┌───────────────────────────────────────────────┐
        # │         BOTTLENECK                            │
        # └───────────────────────────────────────────────┘
        x = self.bottleneck(x)                                      # (N, 512, H/8, W/8)

        # ┌───────────────────────────────────────────────┐
        # │           DECODER BLOCKS (UPSAMPLING)         │
        # └───────────────────────────────────────────────┘

        # --- First Decoder Block (was Second) ---
        g = self.upsample(x)                                        # (N, 512, H/4, W/4)
        x_skip3 = conv3                                             # (N, 256, H/4, W/4)
        att3 = self.att3(g=g, x=x_skip3)                            # (N, 256, H/4, W/4)
        x = torch.cat([g, att3], dim=1)                             # (N, 512 + 256, H/4, W/4)
        x = self.dconv_up3(x)                                       # (N, 256, H/4, W/4)

        # --- Second Decoder Block (was Third) ---
        g = self.upsample(x)                                        # (N, 256, H/2, W/2)
        x_skip2 = conv2                                             # (N, 128, H/2, W/2)
        att2 = self.att2(g=g, x=x_skip2)                            # (N, 128, H/2, W/2)
        x = torch.cat([g, att2], dim=1)                             # (N, 256 + 128, H/2, W/2)
        x = self.dconv_up2(x)                                       # (N, 128, H/2, W/2)

        # --- Third Decoder Block (was Fourth) ---
        g = self.upsample(x)                                        # (N, 128, H, W)
        x_skip1 = conv1                                             # (N, 64, H, W)
        att1 = self.att1(g=g, x=x_skip1)                            # (N, 64, H, W)
        x = torch.cat([g, att1], dim=1)                             # (N, 128 + 64, H, W)
        x = self.dconv_up1(x)                                       # (N, 64, H, W)

        # --- Final Output Layer ---
        out = self.conv_last(x)                                     # (N, out_channels, H, W)

        return out

In [ ]:
class CombinedLoss(nn.Module):
    """
    Combination of BCE, Dice, and boundary loss for clean edges
    """
    def __init__(self, bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2):
        super().__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.boundary_weight = boundary_weight
        self.bce = nn.BCEWithLogitsLoss()

    def dice_loss(self, pred, target, smooth=1e-6):
        pred = torch.sigmoid(pred)
        intersection = (pred * target).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
        dice = (2. * intersection + smooth) / (union + smooth)
        return 1 - dice.mean()

    def boundary_loss(self, pred, target):
        """
        Penalize predictions at boundaries more heavily
        """
        pred = torch.sigmoid(pred)

        # Compute gradient of target (boundaries)
        kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
        kernel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)

        # Detect edges in target
        edge_x = F.conv2d(target, kernel_x, padding=1)
        edge_y = F.conv2d(target, kernel_y, padding=1)
        edges = torch.sqrt(edge_x**2 + edge_y**2)
        edges = (edges > 0.5).float()

        # Weight loss by edges (penalize errors at boundaries more)
        boundary_error = F.binary_cross_entropy(pred, target, reduction='none')
        boundary_error = boundary_error * (1 + 5 * edges)  # 5x penalty at edges

        return boundary_error.mean()

    def forward(self, pred, target):
        bce = self.bce(pred, target)
        dice = self.dice_loss(pred, target)
        boundary = self.boundary_loss(pred, target)

        total = (self.bce_weight * bce +
                self.dice_weight * dice +
                self.boundary_weight * boundary)
        return total

In [ ]:
# Model initialisation

model= UNet(in_channels=3, out_channels=1).to(DEVICE)
BCE_loss = nn.BCEWithLogitsLoss()
criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
def calculate_dice_iou(inputs, targets, smooth=1e-7):
    """
    Calculates Dice and IoU scores for a batch.
    Assumes inputs are binary predictions (0.0 or 1.0) and targets are binary masks.
    """
    # Flatten tensors
    inputs = inputs.view(-1)
    targets = targets.view(-1)

    # Calculate intersection
    intersection = (inputs * targets).sum()

    # Calculate Dice Score
    total_sum = inputs.sum() + targets.sum()
    dice = (2. * intersection + smooth) / (total_sum + smooth)

    # Calculate IoU (Jaccard Index)
    union = total_sum - intersection
    iou = (intersection + smooth) / (union + smooth)

    return dice.item(), iou.item()


In [ ]:

# Training
best_val_dice = -1  # Initialize best_val_dice to a low value

for epoch in range(NUM_EPOCHS):
  train_loss=0

  model.train()
  for images, mask in train_loader:
    images=images.to(DEVICE)
    mask=mask.to(DEVICE)
    optimizer.zero_grad()
    output=model(images)
    # bce_loss=BCE_loss(output, mask)
    # dice_loss_value=dice_loss(output, mask)
    # loss=0.5*bce_loss+0.5*dice_loss_value
    loss=criterion(output, mask)

    loss.backward()
    optimizer.step()
    train_loss+=loss.item()

  avg_train_loss = train_loss / len(train_loader)
# validation
  model.eval()
  tatal_loss=0
  total_dice_loss=0
  total_bce_loss=0
  total_dice_score=0
  total_iou_score=0
  total_val_loss=0
  with torch.no_grad():
    for images, mask in val_loader:

      images=images.to(DEVICE)
      mask=mask.to(DEVICE)
      output=model(images)
      # bce = BCE_loss(output, mask)
      # dice_val_loss = dice_loss(output, mask)
      # loss = 0.5 * bce + 0.5 * dice_val_loss
      loss=criterion(output, mask)
      total_val_loss += loss.item()
      probs=torch.sigmoid(output)
      binary_preds = (probs > 0.5).float()
      dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
      total_dice_score += dice_score
      total_iou_score += iou_score

    # Calculate average metrics for the epoch
    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_dice = total_dice_score / len(val_loader)
    avg_val_iou = total_iou_score / len(val_loader)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
          f"Train Loss: {avg_train_loss:.4f}.. "
          f"Val Loss: {avg_val_loss:.4f}.. "
          f"Val Dice: {avg_val_dice:.4f}.. "
          f"Val IoU: {avg_val_iou:.4f}")

    # --- Save the best model ---
    if 'best_val_dice' in locals() and avg_val_dice > best_val_dice:
        best_val_dice = avg_val_dice
        save_path = "/content/drive/MyDrive/RBE595/p3/dataset/model_size_640.pth"
        torch.save(model.state_dict(), save_path)
        print(f"New best model saved to {save_path} with Dice: {avg_val_dice:.4f}")

Epoch 1/10.. Train Loss: 0.1214.. Val Loss: 0.0642.. Val Dice: 0.9204.. Val IoU: 0.8541
New best model saved to /content/drive/MyDrive/RBE595/p3/dataset/model_size_640.pth with Dice: 0.9204
Epoch 2/10.. Train Loss: 0.0700.. Val Loss: 0.0284.. Val Dice: 0.9693.. Val IoU: 0.9411
New best model saved to /content/drive/MyDrive/RBE595/p3/dataset/model_size_640.pth with Dice: 0.9693
Epoch 3/10.. Train Loss: 0.0559.. Val Loss: 0.0262.. Val Dice: 0.9732.. Val IoU: 0.9485
New best model saved to /content/drive/MyDrive/RBE595/p3/dataset/model_size_640.pth with Dice: 0.9732
Epoch 4/10.. Train Loss: 0.0504.. Val Loss: 0.0258.. Val Dice: 0.9715.. Val IoU: 0.9451
Epoch 5/10.. Train Loss: 0.0455.. Val Loss: 0.0245.. Val Dice: 0.9730.. Val IoU: 0.9479
Epoch 6/10.. Train Loss: 0.0426.. Val Loss: 0.0266.. Val Dice: 0.9701.. Val IoU: 0.9425
Epoch 7/10.. Train Loss: 0.0404.. Val Loss: 0.0233.. Val Dice: 0.9752.. Val IoU: 0.9521
New best model saved to /content/drive/MyDrive/RBE595/p3/dataset/model_size_64

In [ ]:
# --- Testing ---
print("\nTraining Finished. Loading best model for testing.")
model.load_state_dict(torch.load('/content/drive/MyDrive/RBE595/p3/dataset/model_12000.pth')) # Load best weights
model.eval()

total_test_dice = 0
total_test_iou = 0

with torch.no_grad():
    for images, mask in test_loader:
        images = images.to(DEVICE)
        mask = mask.to(DEVICE)
        output = model(images)

        probs = torch.sigmoid(output)
        binary_preds = (probs > 0.5).float()

        dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
        total_test_dice += dice_score
        total_test_iou += iou_score

avg_test_dice = total_test_dice / len(test_loader)
avg_test_iou = total_test_iou / len(test_loader)

print(f"--- Final Test Metrics ---")
print(f"Test Dice Score: {avg_test_dice:.4f}")
print(f"Test IoU Score:  {avg_test_iou:.4f}")


Training Finished. Loading best model for testing.
--- Final Test Metrics ---
Test Dice Score: 0.9428
Test IoU Score:  0.8928


In [ ]:
# import torch
# import os
# import torchvision.utils as vutils # Import for saving image grids

# # --- Testing ---
# print("\nTraining Finished. Loading best model for testing.")
# model.load_state_dict(torch.load('UNet_background_attention_1.pth')) # Load best weights
# model.eval()

# total_test_dice = 0
# total_test_iou = 0

# # --- Additions for saving output ---
# output_dir = "test_outputs"
# os.makedirs(output_dir, exist_ok=True)
# print(f"Saving output masks to {output_dir}/")
# # -------------------------------------


# with torch.no_grad():
#     # Add 'i' for enumeration to get unique filenames
#     for i, (images, mask) in enumerate(test_loader):
#         images = images.to(DEVICE)
#         mask = mask.to(DEVICE)
#         output = model(images)

#         probs = torch.sigmoid(output)
#         binary_preds = (probs > 0.5).float()

#         dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#         total_test_dice += dice_score
#         total_test_iou += iou_score

#         # --- Save the output masks ---
#         # This saves the input, ground truth, and prediction for this batch
#         # 'vutils.save_image' automatically creates a grid if your batch size > 1

#         # Save the original input images
#         vutils.save_image(images,
#                           os.path.join(output_dir, f"batch_{i:03d}_input.png"),
#                           normalize=True) # Normalize for proper viewing

#         # Save the ground truth masks
#         vutils.save_image(mask,
#                           os.path.join(output_dir, f"batch_{i:03d}_ground_truth.png"))

#         # Save your model's predictions
#         vutils.save_image(binary_preds,
#                           os.path.join(output_dir, f"batch_{i:03d}_prediction.png"))
#         # -----------------------------

# avg_test_dice = total_test_dice / len(test_loader)
# avg_test_iou = total_test_iou / len(test_loader)

# print(f"--- Final Test Metrics ---")
# print(f"Test Dice Score: {avg_test_dice:.4f}")
# print(f"Test IoU Score:  {avg_test_iou:.4f}")
# Aerial_P3_Unet_2.ipynb_
# [1]
# 11s

# # ┌───────────────────────────────────────────────┐
# # │                    IMPORTS                    │
# # └───────────────────────────────────────────────┘
# import matplotlib.pyplot as plt
# import torch.nn as nn
# import torch.nn.functional as F
# import torch
# import numpy as np
# from torchvision import datasets, transforms
# from torchvision.transforms import v2
# from torch.utils.data import Dataset, DataLoader
# from torch.optim import AdamW
# import zipfile
# from PIL import Image, ImageOps
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# import os
# import torch
# from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision.transforms import v2 as transforms
# from PIL import Image
# # Determine Device for the whole session
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {DEVICE}")

# Using device: cuda

# [2]
# 26s

# Mounted at /content/drive

# [ ]

# # Unzip dataset
# file_path="/content/drive/MyDrive/RBE595/p3/dataset/window_dataset_12000.zip"
# directory_to_extract_to = "/content/dataset" # Define the extraction directory
# with zipfile.ZipFile(file_path, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

# [3]
# 0s

# image_path = "/content/dataset/window_dataset/images/"
# mask_path = "/content/dataset/window_dataset/masks/"

# # List files in the directories to pick one example
# import os
# image_files = sorted(os.listdir(image_path))
# print(len(image_files))
# mask_files = sorted(os.listdir(mask_path))

# Next steps:
# [ ]
# 0s
# [ ]

# # ┌───────────────────────────────────────────────┐
# # │                 HYPERPARAMETERS               │
# # └───────────────────────────────────────────────┘
# NUM_EPOCHS = 10
# BATCH_SIZE = 8 # Reduced batch size to potentially resolve OOM error

# [ ]

# class CustomDataset(Dataset):
#     '''
#     This dataset loads an image and its corresponding segmentation mask
#     using Albumentations for robust augmentation.
#     '''
#     def __init__(self, image_dir, mask_dir, is_train=True):
#         self.image_dir = image_dir
#         self.mask_dir = mask_dir

#         self.image_files = sorted(os.listdir(self.image_dir))
#         self.mask_files = sorted(os.listdir(self.mask_dir))

#         if is_train:
#             # 1. Training transforms (Merged List)
#             self.transform = A.Compose([
#                 # --- Essential (Keep at top) ---
#                 A.Resize(height=320, width=320, p=1.0),

#                 # --- Spatial Augmentations ---
#                 A.HorizontalFlip(p=0.5),
#                 A.VerticalFlip(p=0.5),      # Updated p=0.5
#                 A.Rotate(limit=45, p=0.5),    # ADDED

#                 # --- Color Augmentations ---
#                 A.ToGray(p=0.1), # Kept from previous version
#                 A.RandomBrightnessContrast(p=0.3), # Updated

#                 # --- Noise, Blur, and Edge Augmentations ---
#                 A.GaussNoise(p=0.4),               # Updated
#                 A.GaussianBlur(blur_limit=(3, 7), p=0.4), # ADDED
#                 A.MotionBlur(blur_limit=7, p=0.3),      # ADDED
#                 A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.4), # ADDED

#                 # --- Dropout Augmentation ---
#                 A.CoarseDropout(
#                     max_holes=8, max_height=20, max_width=20, # Updated
#                     min_holes=3, min_height=5, min_width=5,
#                     fill_value=0, p=0.4
#                 ),

#                 # --- Final conversion (Keep at bottom) ---
#                 A.Normalize(
#                     mean=[0.485, 0.456, 0.406],
#                     std=[0.229, 0.224, 0.225]
#                 ),
#                 ToTensorV2() # Converts image and mask to PyTorch tensors
#             ])
#         else:
#             # 2. Validation/Test transforms (Unchanged)
#             self.transform = A.Compose([
#                 A.Resize(height=320, width=320, p=1.0),
#                 A.Normalize(
#                     mean=[0.485, 0.456, 0.406],
#                     std=[0.229, 0.224, 0.225]
#                 ),
#                 ToTensorV2()
#             ])

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_name = self.image_files[idx]
#         mask_name = self.mask_files[idx]

#         img_path = os.path.join(self.image_dir, img_name)
#         mask_path = os.path.join(self.mask_dir, mask_name)

#         image = Image.open(img_path).convert("RGB")
#         image = ImageOps.exif_transpose(image) # Auto-orientation

#         mask = Image.open(mask_path).convert("L")

#         image_np = np.array(image)
#         mask_np = np.array(mask)

#         # Ensure mask is binary (0 or 1)
#         mask_np = (mask_np > 128).astype(np.uint8) * 255 # Assuming mask is grayscale with values > 128 as foreground

#         transformed = self.transform(image=image_np, mask=mask_np)

#         image_tensor = transformed['image']
#         mask_tensor = transformed['mask'].unsqueeze(0).float() / 255.0 # Normalize mask to 0-1

#         return image_tensor, mask_tensor

# # --- Create Train, Validation, and Test DataLoaders ---

# # Define your paths and parameters
# image_path = "/content/dataset/window_dataset/images/"
# mask_path = "/content/dataset/window_dataset/masks/"


# # Define split ratios
# TRAIN_RATIO = 0.8  # 80% for training
# VAL_RATIO = 0.1   # 10% for validation
# TEST_RATIO = 0.1  # 10% for testing

# # Create the full dataset (with training augmentation for now)
# full_dataset = CustomDataset(image_path, mask_path, is_train=True)

# # Calculate split sizes
# dataset_size = len(full_dataset)
# train_size = int(TRAIN_RATIO * dataset_size)
# val_size = int(VAL_RATIO * dataset_size)
# test_size = dataset_size - train_size - val_size  # Remaining goes to test

# print(f"Total dataset size: {dataset_size}")
# print(f"Train size: {train_size}")
# print(f"Validation size: {val_size}")
# print(f"Test size: {test_size}")

# # Split the dataset
# train_dataset, val_dataset, test_dataset = random_split(
#     full_dataset,
#     [train_size, val_size, test_size],
#     generator=torch.Generator().manual_seed(42)  # For reproducibility
# )

# # Update the is_train flag for validation and test datasets
# # Create new dataset instances for val and test without augmentation
# val_dataset_no_aug = CustomDataset(image_path, mask_path, is_train=False)
# test_dataset_no_aug = CustomDataset(image_path, mask_path, is_train=False)

# # Get the same indices from the split
# val_indices = val_dataset.indices
# test_indices = test_dataset.indices

# # Create subsets with no augmentation
# from torch.utils.data import Subset
# val_dataset = Subset(val_dataset_no_aug, val_indices)
# test_dataset = Subset(test_dataset_no_aug, test_indices)

# # Create DataLoaders
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True,  # Shuffle training data
#     num_workers=2,  # Parallel data loading
#     pin_memory=True  # Faster data transfer to GPU
# )

# val_loader = DataLoader(
#     val_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False,  # Don't shuffle validation data
#     num_workers=2,
#     pin_memory=True
# )

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False,  # Don't shuffle test data
#     num_workers=2,
#     pin_memory=True
# )

# # Print DataLoader information
# print("\n" + "="*50)
# print("DATALOADER INFORMATION")
# print("="*50)
# print(f"\nTrain loader: {len(train_loader)} batches")
# print(f"Validation loader: {len(val_loader)} batches")
# print(f"Test loader: {len(test_loader)} batches")

# # Verify shapes
# try:
#     train_images, train_masks = next(iter(train_loader))
#     print(f"\nTrain batch - Images shape: {train_images.shape}, Masks shape: {train_masks.shape}")

#     val_images, val_masks = next(iter(val_loader))
#     print(f"Val batch - Images shape: {val_images.shape}, Masks shape: {val_masks.shape}")

#     test_images, test_masks = next(iter(test_loader))
#     print(f"Test batch - Images shape: {test_images.shape}, Masks shape: {test_masks.shape}")

#     print("\n✓ All DataLoaders created successfully!")
#     print("="*50)
# except Exception as e:
#     print(f"\nError loading batches: {e}")

# Total dataset size: 7000
# Train size: 5600
# Validation size: 700
# Test size: 700

# ==================================================
# DATALOADER INFORMATION
# ==================================================

# Train loader: 700 batches
# Validation loader: 88 batches
# Test loader: 88 batches

# /tmp/ipython-input-2431407332.py:35: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
#   A.CoarseDropout(


# Train batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])
# Val batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])
# Test batch - Images shape: torch.Size([8, 3, 320, 320]), Masks shape: torch.Size([8, 1, 320, 320])

# ✓ All DataLoaders created successfully!
# ==================================================

# [ ]

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # --- Attention Block (as you provided) ---
# class AttentionBlock(nn.Module):
#     def __init__(self, F_g, F_l, F_int):
#         super(AttentionBlock, self).__init__()
#         self.W_g = nn.Sequential(
#             nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(F_int)
#         )
#         self.W_x = nn.Sequential(
#             nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(F_int)
#         )
#         self.psi = nn.Sequential(
#             nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
#             nn.BatchNorm2d(1),
#             nn.Sigmoid()
#         )
#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, g, x):
#         g1 = self.W_g(g)
#         x1 = self.W_x(x)
#         psi = self.relu(g1 + x1)
#         psi = self.psi(psi)
#         return x * psi

# # --- Double Conv Block (as you provided) ---
# def double_conv(in_channels, out_channels):
#     '''
#     Return a Convolutional Block with Two Convolutional Layers with ReLU Activation based on in_channels and
#     out_channels, with a stride of 3 and padding of 1. If input has H and W, then output will have same H and W.
#     '''
#     return nn.Sequential(
#         nn.Conv2d(in_channels, out_channels, 3, padding=1),
#         nn.BatchNorm2d(out_channels),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(out_channels, out_channels, 3, padding=1),
#         nn.BatchNorm2d(out_channels),
#         nn.ReLU(inplace=True)
#     )

# # --- NEW: Attention U-Net Class (Modified) ---
# class UNet(nn.Module):
#     '''
#     Implementation of an Attention U-Net (3-level).
#     '''
#     def __init__(self, in_channels, out_channels):
#         super().__init__()

#         # --- Encoder (3 levels) ---
#         self.dconv_down1 = double_conv(in_channels, 64)
#         self.dconv_down2 = double_conv(64, 128)
#         self.dconv_down3 = double_conv(128, 256)
#         # Removed dconv_down4
#         self.maxpool = nn.MaxPool2d(2)

#         # --- Bottleneck ---
#         # Input from 3rd level (256 channels), output 512
#         self.bottleneck = double_conv(256, 512)

#         # --- Decoder ---
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

#         # --- Attention Blocks ---
#         # Removed att4
#         # F_g=512 (from upsample(bottleneck)), F_l=256 (from conv3)
#         self.att3 = AttentionBlock(F_g=512, F_l=256, F_int=128)
#         # F_g=256 (from dconv_up3), F_l=128 (from conv2)
#         self.att2 = AttentionBlock(F_g=256, F_l=128, F_int=64)
#         # F_g=128 (from dconv_up2), F_l=64 (from conv1)
#         self.att1 = AttentionBlock(F_g=128, F_l=64, F_int=32)

#         # --- Decoder Convolutions ---
#         # Removed dconv_up4
#         # Input: (g=512) + (att3=256) = 768
#         self.dconv_up3 = double_conv(512 + 256, 256)  # 768  -> 256
#         # Input: (g=256) + (att2=128) = 384
#         self.dconv_up2 = double_conv(256 + 128, 128)  # 384  -> 128
#         # Input: (g=128) + (att1=64) = 192
#         self.dconv_up1 = double_conv(128 + 64, 64)   # 192  -> 64

#         # --- Output Layer ---
#         self.conv_last = nn.Conv2d(64, out_channels, 1)

#     def forward(self, x):
#         '''
#         Forward Pass Method for the Attention UNet
#         '''

#         # ┌───────────────────────────────────────────────┐
#         # │         ENCODER BLOCKS (DOWNSAMPLING)         │
#         # └───────────────────────────────────────────────┘
#         conv1 = self.dconv_down1(x)                                 # (N, 64, H, W)
#         x = self.maxpool(conv1)

#         conv2 = self.dconv_down2(x)                                 # (N, 128, H/2, W/2)
#         x = self.maxpool(conv2)

#         conv3 = self.dconv_down3(x)                                 # (N, 256, H/4, W/4)
#         x = self.maxpool(conv3)                                     # (N, 256, H/8, W/8)

#         # Removed 4th Encoder Block

#         # ┌───────────────────────────────────────────────┐
#         # │         BOTTLENECK                            │
#         # └───────────────────────────────────────────────┘
#         x = self.bottleneck(x)                                      # (N, 512, H/8, W/8)

#         # ┌───────────────────────────────────────────────┐
#         # │           DECODER BLOCKS (UPSAMPLING)         │
#         # └───────────────────────────────────────────────┘

#         # --- First Decoder Block (was Second) ---
#         g = self.upsample(x)                                        # (N, 512, H/4, W/4)
#         x_skip3 = conv3                                             # (N, 256, H/4, W/4)
#         att3 = self.att3(g=g, x=x_skip3)                            # (N, 256, H/4, W/4)
#         x = torch.cat([g, att3], dim=1)                             # (N, 512 + 256, H/4, W/4)
#         x = self.dconv_up3(x)                                       # (N, 256, H/4, W/4)

#         # --- Second Decoder Block (was Third) ---
#         g = self.upsample(x)                                        # (N, 256, H/2, W/2)
#         x_skip2 = conv2                                             # (N, 128, H/2, W/2)
#         att2 = self.att2(g=g, x=x_skip2)                            # (N, 128, H/2, W/2)
#         x = torch.cat([g, att2], dim=1)                             # (N, 256 + 128, H/2, W/2)
#         x = self.dconv_up2(x)                                       # (N, 128, H/2, W/2)

#         # --- Third Decoder Block (was Fourth) ---
#         g = self.upsample(x)                                        # (N, 128, H, W)
#         x_skip1 = conv1                                             # (N, 64, H, W)
#         att1 = self.att1(g=g, x=x_skip1)                            # (N, 64, H, W)
#         x = torch.cat([g, att1], dim=1)                             # (N, 128 + 64, H, W)
#         x = self.dconv_up1(x)                                       # (N, 64, H, W)

#         # --- Final Output Layer ---
#         out = self.conv_last(x)                                     # (N, out_channels, H, W)

#         return out

# [ ]

# class CombinedLoss(nn.Module):
#     """
#     Combination of BCE, Dice, and boundary loss for clean edges
#     """
#     def __init__(self, bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2):
#         super().__init__()
#         self.bce_weight = bce_weight
#         self.dice_weight = dice_weight
#         self.boundary_weight = boundary_weight
#         self.bce = nn.BCEWithLogitsLoss()

#     def dice_loss(self, pred, target, smooth=1e-6):
#         pred = torch.sigmoid(pred)
#         intersection = (pred * target).sum(dim=(2, 3))
#         union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
#         dice = (2. * intersection + smooth) / (union + smooth)
#         return 1 - dice.mean()

#     def boundary_loss(self, pred, target):
#         """
#         Penalize predictions at boundaries more heavily
#         """
#         pred = torch.sigmoid(pred)

#         # Compute gradient of target (boundaries)
#         kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
#                                 dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
#         kernel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
#                                 dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)

#         # Detect edges in target
#         edge_x = F.conv2d(target, kernel_x, padding=1)
#         edge_y = F.conv2d(target, kernel_y, padding=1)
#         edges = torch.sqrt(edge_x**2 + edge_y**2)
#         edges = (edges > 0.5).float()

#         # Weight loss by edges (penalize errors at boundaries more)
#         boundary_error = F.binary_cross_entropy(pred, target, reduction='none')
#         boundary_error = boundary_error * (1 + 5 * edges)  # 5x penalty at edges

#         return boundary_error.mean()

#     def forward(self, pred, target):
#         bce = self.bce(pred, target)
#         dice = self.dice_loss(pred, target)
#         boundary = self.boundary_loss(pred, target)

#         total = (self.bce_weight * bce +
#                 self.dice_weight * dice +
#                 self.boundary_weight * boundary)
#         return total

# [ ]

# # Model initialisation

# model= UNet(in_channels=3, out_channels=1).to(DEVICE)
# BCE_loss = nn.BCEWithLogitsLoss()
# criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.3, boundary_weight=0.2)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# [ ]

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# def calculate_dice_iou(inputs, targets, smooth=1e-7):
#     """
#     Calculates Dice and IoU scores for a batch.
#     Assumes inputs are binary predictions (0.0 or 1.0) and targets are binary masks.
#     """
#     # Flatten tensors
#     inputs = inputs.view(-1)
#     targets = targets.view(-1)

#     # Calculate intersection
#     intersection = (inputs * targets).sum()

#     # Calculate Dice Score
#     total_sum = inputs.sum() + targets.sum()
#     dice = (2. * intersection + smooth) / (total_sum + smooth)

#     # Calculate IoU (Jaccard Index)
#     union = total_sum - intersection
#     iou = (intersection + smooth) / (union + smooth)

#     return dice.item(), iou.item()

# [ ]


# # Training
# best_val_dice = -1  # Initialize best_val_dice to a low value

# for epoch in range(NUM_EPOCHS):
#   train_loss=0

#   model.train()
#   for images, mask in train_loader:
#     images=images.to(DEVICE)
#     mask=mask.to(DEVICE)
#     optimizer.zero_grad()
#     output=model(images)
#     # bce_loss=BCE_loss(output, mask)
#     # dice_loss_value=dice_loss(output, mask)
#     # loss=0.5*bce_loss+0.5*dice_loss_value
#     loss=criterion(output, mask)

#     loss.backward()
#     optimizer.step()
#     train_loss+=loss.item()

#   avg_train_loss = train_loss / len(train_loader)
# # validation
#   model.eval()
#   tatal_loss=0
#   total_dice_loss=0
#   total_bce_loss=0
#   total_dice_score=0
#   total_iou_score=0
#   total_val_loss=0
#   with torch.no_grad():
#     for images, mask in val_loader:

#       images=images.to(DEVICE)
#       mask=mask.to(DEVICE)
#       output=model(images)
#       # bce = BCE_loss(output, mask)
#       # dice_val_loss = dice_loss(output, mask)
#       # loss = 0.5 * bce + 0.5 * dice_val_loss
#       loss=criterion(output, mask)
#       total_val_loss += loss.item()
#       probs=torch.sigmoid(output)
#       binary_preds = (probs > 0.5).float()
#       dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#       total_dice_score += dice_score
#       total_iou_score += iou_score

#     # Calculate average metrics for the epoch
#     avg_val_loss = total_val_loss / len(val_loader)
#     avg_val_dice = total_dice_score / len(val_loader)
#     avg_val_iou = total_iou_score / len(val_loader)

#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
#           f"Train Loss: {avg_train_loss:.4f}.. "
#           f"Val Loss: {avg_val_loss:.4f}.. "
#           f"Val Dice: {avg_val_dice:.4f}.. "
#           f"Val IoU: {avg_val_iou:.4f}")

#     # --- Save the best model ---
#     if 'best_val_dice' in locals() and avg_val_dice > best_val_dice:
#         best_val_dice = avg_val_dice
#         save_path = "/content/drive/MyDrive/RBE595/p3/dataset/model_12000.pth"
#         torch.save(model.state_dict(), save_path)
#         print(f"New best model saved to {save_path} with Dice: {avg_val_dice:.4f}")

# Epoch 1/10.. Train Loss: 0.1810.. Val Loss: 0.0342.. Val Dice: 0.9704.. Val IoU: 0.9427
# New best model saved with Dice: 0.9704
# Epoch 2/10.. Train Loss: 0.1020.. Val Loss: 0.0277.. Val Dice: 0.9736.. Val IoU: 0.9487
# New best model saved with Dice: 0.9736
# Epoch 3/10.. Train Loss: 0.0860.. Val Loss: 0.0306.. Val Dice: 0.9674.. Val IoU: 0.9372
# Epoch 4/10.. Train Loss: 0.0833.. Val Loss: 0.0214.. Val Dice: 0.9782.. Val IoU: 0.9574
# New best model saved with Dice: 0.9782
# Epoch 5/10.. Train Loss: 0.0747.. Val Loss: 0.0210.. Val Dice: 0.9785.. Val IoU: 0.9580
# New best model saved with Dice: 0.9785
# Epoch 6/10.. Train Loss: 0.0692.. Val Loss: 0.0162.. Val Dice: 0.9831.. Val IoU: 0.9668
# New best model saved with Dice: 0.9831
# Epoch 7/10.. Train Loss: 0.0637.. Val Loss: 0.0181.. Val Dice: 0.9805.. Val IoU: 0.9618
# Epoch 8/10.. Train Loss: 0.0586.. Val Loss: 0.0169.. Val Dice: 0.9818.. Val IoU: 0.9643
# Epoch 9/10.. Train Loss: 0.0553.. Val Loss: 0.0233.. Val Dice: 0.9718.. Val IoU: 0.9454
# Epoch 10/10.. Train Loss: 0.0552.. Val Loss: 0.0166.. Val Dice: 0.9829.. Val IoU: 0.9664

# [ ]

# # --- Testing ---
# print("\nTraining Finished. Loading best model for testing.")
# model.load_state_dict(torch.load('UNet_background_attention_1.pth')) # Load best weights
# model.eval()

# total_test_dice = 0
# total_test_iou = 0

# with torch.no_grad():
#     for images, mask in test_loader:
#         images = images.to(DEVICE)
#         mask = mask.to(DEVICE)
#         output = model(images)

#         probs = torch.sigmoid(output)
#         binary_preds = (probs > 0.5).float()

#         dice_score, iou_score = calculate_dice_iou(binary_preds, mask)
#         total_test_dice += dice_score
#         total_test_iou += iou_score

# avg_test_dice = total_test_dice / len(test_loader)
# avg_test_iou = total_test_iou / len(test_loader)

# print(f"--- Final Test Metrics ---")
# print(f"Test Dice Score: {avg_test_dice:.4f}")
# print(f"Test IoU Score:  {avg_test_iou:.4f}")


# Training Finished. Loading best model for testing.
# --- Final Test Metrics ---
# Test Dice Score: 0.9829
# Test IoU Score:  0.9665

# [ ]


# Training Finished. Loading best model for testing.
# Saving output masks to test_outputs/
# --- Final Test Metrics ---
# Test Dice Score: 0.9829
# Test IoU Score:  0.9665

# Finished testing. All output images saved to 'test_outputs'.

# [ ]
# Colab paid products - Cancel contracts here


# print(f"\nFinished testing. All output images saved to '{output_dir}'.")